In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statsmodels.formula.api import logit
import statsmodels.formula.api as sm
import warnings
warnings.filterwarnings('ignore')

/Users/christams/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [125]:
Pro_TE_Path = 'Data/TE_data_Trail.csv'

data = pd.read_csv(Pro_TE_Path)

players = pd.read_csv('Data/Names.csv')

In [126]:
pd.set_option('display.max_columns', None)

In [127]:
data = data.fillna(0)
data = data.drop(columns=["Unnamed: 0", "Pos_"])

In [128]:
def staus(data):
    different_staus_options = []
    temp = data.drop(columns=["Name_", "Unnamed: 2_level_0_Date", 
                              "Unnamed: 6_level_0_Tm", "Unnamed: 7_level_0_Unnamed: 7_level_1",
                              "Unnamed: 9_level_0_Result", "Unnamed: 8_level_0_Opp" ])
    for x in temp.columns:
        statuses = data[x].unique()
        for w in statuses:
            try:
                float(w)
            except:
                try:
                    float(w[:-1])
                except:
                    different_staus_options.append(w)
                    #print(w)
    return different_staus_options

In [129]:
for index, name in enumerate(data["Name_"]):
    if name == 0:
        data["Name_"][index] = data["Name__"][index]
        data["Name__"][index] = 0
data = data.drop(columns = ["Name__"])

In [130]:
stats = staus(data)

In [131]:
set_res = set(stats) 
list_res = (list(set_res))
values_to_replace = []
 
for item in list_res: 
    print(item)
    values_to_replace.append(item)

COVID-19 List
Physically Unable to Perform
Did Not Play
Exempt List
Injured Reserve
Non-Football Injury
*
Suspended
Inactive


In [132]:
for x in values_to_replace:
    data[x] = 0

for w in data.columns:
    for index, z in enumerate(data[w]):
        if values_to_replace.count(z) > 0:
            #print(z, index)
            data[z][index] = 1

In [133]:
for x in values_to_replace:
    data = data.replace(x, 0)

In [134]:
for x in data.columns:
    length = len("Unnamed: ")
    if x[:length] == "Unnamed: ":
        real_col = x.split("_")
        data = data.rename(columns={ x : real_col[-1]})
        #print(real_col[-1])
        

In [135]:
#Delete Duplicate columns
data = data.loc[:,~data.columns.duplicated()].copy()

In [136]:
def result_maker(data):
    data["Win?"] = 0
    data["Lose?"] = 0
    data["Tie?"] = 0
    data["Score-not-counted"] = 0
    data["Tm-Score"] = 0
    data["Opp-Score"] = 0
    for index, x in enumerate(data["Result"]):
        if x[0] == "W" or x[0] == "L" or x[0] == "T":
            if x[0] == "W":
                data["Win?"][index] = 1
            elif x[0] == "L":
                data["Lose?"][index] = 1
            else:
                data["Tie?"][index] = 1
            ind_num = x[1:].split("-")
            if len(ind_num) == 2:
                data["Tm-Score"][index] = ind_num[0]
                data["Opp-Score"][index] = ind_num[1]
        else:
            data["Score-not-counted"][index] = 1

In [137]:
result_maker(data)
data = data.drop(columns=["Result"])

In [138]:
def seperate_date(data):
    data["Day"] = 0
    data["Month"] = 0
    data["No-Date"] = 0
    
    for index, x in enumerate(data["Date"]):
        Year_Month_Day = x.split("-")
        if len(Year_Month_Day) > 1:
            Month = Year_Month_Day[1]
            Day = Year_Month_Day[2]
        
            data["Day"][index] = Day
            data["Month"][index] = Month
        else:
            data["No-Date"][index] = 1
    

In [139]:
seperate_date(data)
data = data.drop(columns=["Date"])

In [140]:
def convert_percentage(data):
    for col in data.columns:
        for index, x in enumerate(data[col]):
            try:
                per = x[-1]
                if per == "%": 
                    data[col] = x[:-1]
            except:
                pass

In [141]:
convert_percentage(data)

In [142]:
def convert_to_number(data):
    temp = data.drop(columns = ["Name_", "Tm", "1", "Opp"])
    didnt_work = {}
    
    for col in temp.columns:
        #print(col)
        for x in data[col]:
            try:
                data[col] = x.astype(float)
            except:
                pass
            
        

In [143]:
convert_to_number(data)

In [144]:
def eliminate_Useless(data, num = 1, drop = False):
    limited = []
    for x in data.columns:
        amount = len(data[x].unique())
        if amount <= num:
            limited.append(x)
            print(x, amount, data[x].unique())
    if drop:
        data = data.drop(columns = limited)

In [145]:
eliminate_Useless(data, num = 5, drop = False)

Def. Snaps_Pct 1 ['0']
Fumbles_TD 3 ['0' 0 '1']
Kick Returns_TD 3 [0 '0' '1']
Off. Snaps_Pct 1 ['82']
Punt Returns_TD 3 [0 '0' '1']
Punting_Blck 2 [0 '0']
Punting_In20 3 [0 '0' '1']
Punting_In20% 1 ['0.0']
Punting_NY/P 5 [0 '13.0' '24.0' '15.0' '17.3']
Punting_Pnt 4 [0 '0' '1' '3']
Punting_RetYds 2 [0 '0']
Punting_TB 3 [0 '0' '1']
Punting_TB% 1 ['100.0']
Punting_Y/P 4 [0 '33.00' '24.00' '15.00']
Receiving_Ctch% 1 ['60.0']
ST Snaps_Pct 1 ['12']
Scoring_FG% 4 [  0.  100.   33.3  28.6]
Scoring_FGA 4 [0. 1. 3. 7.]
Scoring_FGM 3 [0. 1. 2.]
Scoring_Sfty 2 [0 '1']
Scoring_XP% 5 [  0.   75.  100.   90.9  50. ]
Scoring_XPA 5 [ 0.  4.  5.  2. 11.]
GS 1 [0]
Status 1 [0]
1 2 ['@' 0]
COVID-19 List 2 [0 1]
Physically Unable to Perform 2 [0 1]
Did Not Play 2 [0 1]
Exempt List 2 [0 1]
Injured Reserve 2 [0 1]
Non-Football Injury 2 [0 1]
* 2 [1 0]
Suspended 2 [0 1]
Inactive 2 [0 1]
Win? 2 [0 1]
Lose? 2 [1 0]
Tie? 2 [0 1]
Score-not-counted 2 [0 1]
No-Date 2 [0 1]


In [146]:
data = data.drop(columns=["Def. Snaps_Pct",
                         "Off. Snaps_Pct",
                         "Receiving_Ctch%",
                         "ST Snaps_Pct",
                         "Punting_TB%",
                         "GS",
                         "Sk",
                         "Status"])

In [147]:
temp = data.drop(columns = {"Name_", "Tm", "1", "Opp"})
for x in temp.columns:
    if temp[x].dtype == object:
        try:
            data[x] = pd.to_numeric(data[x])
        except:
            print(x)

In [148]:
dk_points = pd.read_csv('Data/WeeklyPoints2014-2021.csv', delimiter = ";")
dk_points

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,DK_points,DK_salary
0,17,2014,1426,"Smith, Geno",QB,nyj,a,mia,30.12,5000
1,17,2014,1408,"Griffin III, Robert",QB,was,h,dal,25.34,5700
2,17,2014,1252,"Rodgers, Aaron",QB,gnb,h,det,24.44,8600
3,17,2014,1302,"Flacco, Joe",QB,bal,h,cle,23.58,6200
4,17,2014,1225,"Manning, Eli",QB,nyg,h,phi,23.16,7100
...,...,...,...,...,...,...,...,...,...,...
55003,1,2021,7002,Atlanta,Def,atl,h,phi,0,2000
55004,1,2021,7006,Chicago,Def,chi,a,lar,0,2800
55005,1,2021,7030,Tennessee,Def,ten,h,ari,0,2300
55006,1,2021,7014,Jacksonville,Def,jac,a,hou,-3,2700


In [149]:
title = ['Week', 'Year', 'GID', "DK_points", "DK_salary"]
index_to_remove = []
for name in title:
    for x in range(len(dk_points[name])):
        try:
            dk_points[name][x] = float(dk_points[name][x])
        except ValueError:
            dk_points[name][x] = 0  
            index_to_remove.append(x)
        

In [150]:
# Replace 'DK points' with NaN (or another value of your choice)
dk_points['Week'] = dk_points['Week'].replace('Week', float('nan'))
dk_points['Week'] = dk_points['Week'].astype(float)


dk_points['Year'] = dk_points['Year'].replace('Year', float('nan'))
dk_points['Year'] = dk_points['Year'].astype(float)

dk_points['DK_points'] = dk_points['DK_points'].replace('DK_points', float('nan'))
dk_points['DK_points'] = dk_points['DK_points'].astype(float)

dk_points['DK_salary'] = dk_points['DK_salary'].replace('DK_salary', float('nan'))
dk_points['DK_salary'] = dk_points['DK_salary'].astype(float)

In [151]:
def get_points(data):
    data["Dk_Points"] = 0
    data["Salary"] = 0
    
    for index, row in dk_points.iterrows():
        Name = row["Name"]
        Week = row["Week"]
        Year = row["Year"]
        
        Index = data[(data["Name_"] == Name) & (data["Year"] == float(Year)) & (data["Week"] == int(Week))].index
        
        data["Dk_Points"][Index] = row["DK_points"]
        data["Salary"][Index] = row["DK_salary"]
    return data

In [152]:
get_points(data)

,Def Interceptions_Int,Def Interceptions_PD,Def Interceptions_TD,Def Interceptions_Yds,Def. Snaps_Num,Fumbles_FF,Fumbles_FL,Fumbles_FR,Fumbles_Fmb,Fumbles_TD,Fumbles_Yds,Kick Returns_Rt,Kick Returns_TD,Kick Returns_Y/Rt,Kick Returns_Yds,Name_,Off. Snaps_Num,Passing_AY/A,Passing_Att,Passing_Cmp,Passing_Cmp%,Passing_Int,Passing_Rate,Passing_Sk,Passing_TD,Passing_Y/A,Passing_Yds,Passing_Yds.1,Punt Returns_Ret,Punt Returns_TD,Punt Returns_Y/R,Punt Returns_Yds,Punting_Blck,Punting_In20,Punting_In20%,Punting_NY/P,Punting_Net,Punting_Pnt,Punting_RetYds,Punting_TB,Punting_Y/P,Punting_Yds,Receiving_Rec,Receiving_TD,Receiving_Tgt,Receiving_Y/R,Receiving_Y/Tgt,Receiving_Yds,Rushing_Att,Rushing_TD,Rushing_Y/A,Rushing_Yds,ST Snaps_Num,Scoring_2PM,Scoring_FG%,Scoring_FGA,Scoring_FGM,Scoring_Pts,Scoring_Sfty,Scoring_TD,Scoring_XP%,Scoring_XPA,Scoring_XPM,Tackles_Ast,Tackles_Comb,Tackles_QBHits,Tackles_Solo,Tackles_TFL,Rk,Year,G#,Week,Age,Tm,1,Opp,COVID-19 List,Physically Unable to Perform,Did Not Play,Exempt List,Injured Reserve,Non-Football Injury,*,Suspended,Inactive,Win?,Lose?,Tie?,Score-not-counted,Tm-Score,Opp-Score,Day,Month,No-Date,Dk_Points,Salary
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0,"Fleener, Coby",54.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0,0,0,0.0,0,0,0,0.0,0.0,0,0,0,0,0.0,0,6.0,0.0,10.0,13.67,8.20,82.0,0,0,0.0,0,4.0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2012.0,1.0,1.0,23.355,IND,@,CHI,0,0,0,0,0,0,1,0,0,0,1,0,0,21,41,9,9,0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0,"Fleener, Coby",50.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0,0,0,0.0,0,0,0,0.0,0.0,0,0,0,0,0.0,0,2.0,0.0,4.0,8.00,4.00,16.0,0,0,0.0,0,10.0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2012.0,2.0,2.0,23.362,IND,0,MIN,0,0,0,0,0,0,1,0,0,1,0,0,0,23,20,16,9,0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0,"Fleener, Coby",43.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0,0,0,0.0,0,0,0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,2.0,0.00,0.00,0.0,0,0,0.0,0,4.0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2012.0,3.0,3.0,24.003,IND,0,JAX,0,0,0,0,0,0,1,0,0,0,1,0,0,17,22,23,9,0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0,"Fleener, Coby",53.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0,0,0,0.0,0,0,0,0.0,0.0,0,0,0,0,0.0,0,5.0,0.0,9.0,8.20,4.56,41.0,0,0,0.0,0,5.0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2012.0,4.0,5.0,24.017,IND,0,GNB,0,0,0,0,0,0,1,0,0,1,0,0,0,30,27,7,10,0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0,"Fleener, Coby",35.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0,0,0,0.0,0,0,0,0.0,0.0,0,0,0,0,0.0,0,4.0,0.0,6.0,10.50,7.00,42.0,0,0,0.0,0,3.0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2012.0,5.0,6.0,24.024,IND,@,NYJ,0,0,0,0,0,0,1,0,0,0,1,0,0,9,35,14,10,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0,"Ellis, Alex",0.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0,0,0,0.0,0,0,0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.00,0.00,0.0,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,2021.0,14.0,15.0,0.000,ARI,@,DET,0,0,1,0,0,0,0,0,0,0,1,0,0,12,30,19,12,0,0.0,0.0
29181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0,"Ellis, Alex",0.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0,0,0,0.0,0,0,0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.00,0.00,0.0,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,2021.0,15.0,16.0,0.000,ARI,0,IND,0,0,1,0,0,0,0,0,0,0,1,0,0,16,22,25,12,0,0.0,0.0
29182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0,"Ellis, Alex",0.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0,0,0,0.0,0,0,0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.00,0.00,0.0,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,2021.0,16.0,17.0,0.000,ARI,@,DAL,0,

In [153]:
#names = pd.DataFrame(data={"Name_":data["Name_"].unique()})
#print(data["Name_"].unique(), "\n", data["Name_"])
#names.to_csv('filtered_data_RB.csv', index=False, sep=";")

In [156]:
def filter_for_Prep(data):
    
    filtered_data = data[data["Year"] >= 2013]
    filtered_data = filtered_data.drop(columns = ["Rk", "G#"])
    filtered_data = filtered_data.loc[:,~data.T.duplicated(keep='first')]
    do_not_drop = ['Injured Reserve','Non-Football Injury','Suspended','Exempt List','Did Not Play','Physically Unable to Perform','Inactive','COVID-19 List','Tie?','Score-not-counted']
    eliminated = []
    eliminated_value = []
    for col in filtered_data.columns:
        num = (filtered_data[col] == 0).sum()/len(filtered_data)
        
        if num > 0.89 and do_not_drop.count(col) == 0:
            eliminated.append(col)
            eliminated_value.append(num)
            filtered_data = filtered_data.drop(columns=[col])
        
        
        
            
    return filtered_data, eliminated

In [157]:
filtered_data, eliminated = filter_for_Prep(data)

In [158]:
heights_and_weights = pd.read_csv('Data/data_RBs.csv', sep = ";")

heights_and_weights["Feet"] = 0
heights_and_weights["Inches"] = 0
heights_and_weights["Weight(lbs)"] = 0

index = 0
for x in heights_and_weights["Info"]:
    feet = x[0]
    inch = x[2]
    weight = x[5:8]
    
    heights_and_weights["Feet"][index] = feet
    heights_and_weights["Inches"][index] = inch
    heights_and_weights["Weight(lbs)"][index] = weight
    index += 1

In [159]:
index = 0
filtered_data = filtered_data.reset_index(drop=True)
filtered_data["Feet"] = 0
filtered_data["Inches"] = 0
filtered_data["Weight(lbs)"] = 0

dumb_players_with_no_weight = []
dumb_players_with_no_weight_index = []
for index, x in enumerate(filtered_data["Name_"]):
    try:
        indices = heights_and_weights[heights_and_weights['Name'] == x].index[0]

        filtered_data["Feet"][index] = heights_and_weights["Feet"][indices]
        filtered_data["Inches"][index] = heights_and_weights["Inches"][indices]
        filtered_data["Weight(lbs)"][index] = heights_and_weights["Weight(lbs)"][indices]
    except:
        dumb_players_with_no_weight.append(x)
        dumb_players_with_no_weight_index.append(index)
        
    index += 1

    
filtered_data.drop(labels = dumb_players_with_no_weight_index, axis = 0)
filtered_data = filtered_data.reset_index(drop=True)

In [163]:
filtered_data[(filtered_data["Name_"] == "Kelce, Travis") & (filtered_data["Year"] == 2021)]

,Name_,Off. Snaps_Num,Receiving_Rec,Receiving_Tgt,Receiving_Y/R,Receiving_Y/Tgt,Receiving_Yds,ST Snaps_Num,Year,Week,Age,Tm,1,Opp,COVID-19 List,Physically Unable to Perform,Did Not Play,Exempt List,Injured Reserve,Non-Football Injury,*,Suspended,Inactive,Win?,Lose?,Tie?,Score-not-counted,Tm-Score,Opp-Score,Day,Month,Dk_Points,Salary,Feet,Inches,Weight(lbs)
320,"Kelce, Travis",55.0,6.0,7.0,12.67,10.86,76.0,0.0,2021.0,1.0,31.342,KAN,0,CLE,0,0,0,0,0,0,1,0,0,1,0,0,0,33,29,12,9,25.6,8300.0,0,0,0
321,"Kelce, Travis",44.0,7.0,8.0,15.57,13.63,109.0,0.0,2021.0,2.0,31.349,KAN,@,BAL,0,0,0,0,0,0,1,0,0,0,1,0,0,35,36,19,9,26.9,8500.0,0,0,0
322,"Kelce, Travis",69.0,7.0,11.0,14.86,9.45,104.0,0.0,2021.0,3.0,31.356,KAN,0,LAC,0,0,0,0,0,0,1,0,0,0,1,0,0,24,30,26,9,20.4,8200.0,0,0,0
323,"Kelce, Travis",59.0,4.0,6.0,5.75,3.83,23.0,0.0,2021.0,4.0,31.363,KAN,@,PHI,0,0,0,0,0,0,1,0,0,1,0,0,0,42,30,3,10,6.3,8100.0,0,0,0
324,"Kelce, Travis",75.0,6.0,10.0,9.50,5.70,57.0,0.0,2021.0,5.0,32.005,KAN,0,BUF,0,0,0,0,0,0,1,0,0,0,1,0,0,20,38,10,10,17.7,7600.0,0,0,0
325,"Kelce, Travis",67.0,8.0,11.0,12.38,9.00,99.0,0.0,2021.0,6.0,32.012,KAN,@,WAS,0,0,0,0,0,0,1,0,0,1,0,0,0,31,13,17,10,17.9,7000.0,0,0,0
326,"Kelce, Travis",53.0,7.0,12.0,9.29,5.42,65.0,0.0,2021.0,7.0,32.019,KAN,@,TEN,0,0,0,0,0,0,1,0,0,0,1,0,0,3,27,24,10,13.5,7600.0,0,0,0
327,"Kelce, Travis",74.0,4.0,6.0,6.75,4.50,27.0,0.0,2021.0,8.0,32.027,KAN,0,NYG,0,0,0,0,0,0,1,0,0,1,0,0,0,20,17,1,11,5.7,7500.0,0,0,0
328,"Kelce, Travis",56.0,5.0,8.0,13.60,8.50,68.0,0.0,2021.0,9.0,32.033,KAN,0,GNB,0,0,0,0,0,0,1,0,0,1,0,0,0,13,7,7,11,17.8,7000.0,0,0,0
329,"Kelce, Travis",58.0,8.0,10.0,14.88,11.90,119.0,0.0,2021.0,10.0,32.040,KAN,@,LVR,0,0,0,0,0,0,1,0,0,1,0,0,0,41,14,14,11,22.9,6900.0,0,0,0


In [164]:
Fan_Points_2013_2022 = pd.read_csv('Data/Fantasy_Points_2013-2022 - Sheet1.csv')
Fan_Points_2013_2022.TEAM = Fan_Points_2013_2022.TEAM.replace("LV", "LVR").replace("JAC", "JAX").replace("SF", "SFO").replace("TB", "TAM").replace("GB", "GNB").replace("KC", "KAN").replace("NO", "NOR").replace("NE", "NWE")

In [165]:
Fan_Points_2013_2022 = Fan_Points_2013_2022[Fan_Points_2013_2022["POS"] == "TE"].reset_index(drop=True)
Fan_Points_2013_2022["Convert_Name"] = 0


for index, name in enumerate(Fan_Points_2013_2022["PLAYER"]):
    name_seg = name.split(" ")
    if len(name_seg) == 2:
        Fan_Points_2013_2022["Convert_Name"][index] = name_seg[1] + ", " + name_seg[0]
    else:
        Fan_Points_2013_2022["Convert_Name"][index] = name_seg[1] + " " + name_seg[2] + ", " + name_seg[0]

In [166]:
def place_2022_scoring(filtered_data, Fan_Points_2013_2022):
    name_options = list(filtered_data["Name_"].unique())
    idoit = []
    accounted = 0
    amount = 1
    for index, row in Fan_Points_2013_2022.iterrows():
        name = row["Convert_Name"]
        week = 1
        if row["Year"] >= 2021:
            Games = row[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']]
        else:
            Games = row[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17']]
        if name in name_options:
            #print(name)
            for index, col in enumerate(Games.keys(), start=1):
                if col in Games:
                    try:
                        points = float(Games[col])
                    except:
                        points = 0
                        accounted += 1
                        
                    filtered_data.loc[(filtered_data["Year"] == int(row["Year"])) & (filtered_data["Name_"] == name) & (filtered_data["Week"] == week), 'Dk_Points'] = points
                amount += 1
                week += 1
        else:
            idoit.append(name)
    print(len(idoit)/len(list(filtered_data["Name_"].unique())) * 100, "% of the points sheet are not in", "\n", accounted/amount * 100, "% was accounted for")
    filtered_data["Dk_Points"] = filtered_data["Dk_Points"].astype(float)
    return idoit

In [167]:
idoit = place_2022_scoring(filtered_data, Fan_Points_2013_2022)


2.1604938271604937 % of the points sheet are not in 
 35.49789339931787 % was accounted for


In [168]:
temp = pd.DataFrame(data = {"Name_" : idoit})
idoit = list(temp["Name_"].unique())
idoit

['Pierce Jr., Chris',
 'Allen, Austin',
 'Hausmann, Jake',
 'Moore, Briley',
 'Jones, Charles',
 'Franks, Jordan']

In [172]:
# more_forgotten_RB = {
#     "Hockenson, T.J." : "https://www.pro-football-reference.com/players/H/HockTJ00/gamelog/",
#     "Woods, Jelani" : "https://www.pro-football-reference.com/players/W/WoodJe01/gamelog/",
#     "Parkinson, Colby" : "https://www.pro-football-reference.com/players/P/ParkCo02/gamelog/",
#     "Smith Jr., Irv" : "https://www.pro-football-reference.com/players/S/SmitIr01/gamelog/",
#     "Smith, Jonnu" : "https://www.pro-football-reference.com/players/S/SmitJo01/gamelog/",
#     "Fortson, Jody" : "https://www.pro-football-reference.com/players/F/FortJo01/gamelog/",
#     "Hopkins, Brycen" : "https://www.pro-football-reference.com/players/H/HopkBr01/gamelog/",
#     "Davis, Tyler" : "https://www.pro-football-reference.com/players/H/HopkBr01/gamelog/",
#     "Williams, Maxx" : "https://www.pro-football-reference.com/players/W/WillMa04/gamelog/",
#     "Anderson, Stephen" : "https://www.pro-football-reference.com/players/A/AndeSt01/gamelog/",
#     "Harris, Jacob" : "https://www.pro-football-reference.com/players/H/HarrJa03/gamelog/",
#     "Pinkney, Jared" : "https://www.pro-football-reference.com/players/P/PinkJa02/gamelog/",
#     "Cook, Jared" : "https://www.pro-football-reference.com/players/C/CookJa02/gamelog/",
#     "Hollister, Jacob" : "https://www.pro-football-reference.com/players/H/HollJa03/gamelog/",
#     "Brown, Daniel" : "https://www.pro-football-reference.com/players/B/BrowDa04/gamelog/",
#     "Harris, Demetrius" : "https://www.pro-football-reference.com/players/H/HarrDe03/gamelog/",
#     "Daniels, Darrell" : "https://www.pro-football-reference.com/players/D/DaniDa02/gamelog/",
#     "Thompson, Colin" : "https://www.pro-football-reference.com/players/T/ThomCo03/gamelog/",
#     "Harris, Clark" : "https://www.pro-football-reference.com/players/H/HarrCl20/gamelog/",
#     "Berry, Rashod" : "https://www.pro-football-reference.com/players/B/BerrRa01/gamelog/",
#     "Reed, Jordan" : "https://www.pro-football-reference.com/players/R/ReedJo02/gamelog/",
#     "Thomas, Jordan" : "https://www.pro-football-reference.com/players/T/ThomJo05/gamelog/",
#     "Graham, Jaeden" : "https://www.pro-football-reference.com/players/G/GrahJa01/gamelog/",
#     "Bryant, Hunter" : "https://www.pro-football-reference.com/players/B/BryaHu02/gamelog/",
#     "Miller, Bruce" : "https://www.pro-football-reference.com/players/M/MillBr02/gamelog/",
#     "Baugh, Marcus" : "https://www.pro-football-reference.com/players/B/BaugMa02/gamelog/",
#     "O'Leary, Nick" : "https://www.pro-football-reference.com/players/O/OLeaNi00/gamelog/",
#     "Perkins, Joshua" : "https://www.pro-football-reference.com/players/P/PerkJo02/gamelog/",
#     "Ellis, Alex" : "https://www.pro-football-reference.com/players/E/ElliAl01/gamelog/"
    
# } 

In [173]:
# last = []
# first = []

# for x in idoit:
#     sep = x.split(", ")
#     last.append(sep[0])
#     first.append(sep[1])

In [174]:
# forgotten = pd.DataFrame(data={"Name_":idoit, "First Name": first, "Last Name": last})

# forgotten.to_csv('Forgotten_TEs.csv', index=False, sep=";")

In [191]:
Offesive_Team_Stats = pd.read_csv('Data/NFL Team Season Stats - Offense Reg Season.csv', thousands=',')
Defensive_Team_Stats = pd.read_csv('Data/NFL Team Season Stats - Defense Reg Season.csv', thousands=',')
filtered_data["Tm"] = filtered_data["Tm"].replace("STL", "NOR").replace("OAK", "LVR").replace("SDG", "LAC")

In [192]:
def offensive_scoring_creation(filtered_data):
    filtered_data["Last_Off_Total_YDS"] = 0
    filtered_data["Last_Off_Total_YDS/G"] = 0
    filtered_data["Last_Off_Passing_YDS"] = 0
    filtered_data["Last_Off_Passing_YDS/G"] = 0
    filtered_data["Last_Off_Rushing_YDS"] = 0
    filtered_data["Last_Off_Rushing_YDS/G"] = 0
    filtered_data["Last_Off_Points_PTS"] = 0
    filtered_data["Last_Off_PointsPTS/G"] = 0
    for x in filtered_data.index:
        Team = filtered_data.loc[x]["Tm"]
        Year = filtered_data.loc[x]["Year"]
        row = Offesive_Team_Stats[(Offesive_Team_Stats["TEAM"] == Team) & (Offesive_Team_Stats["Year"] == Year)]

        # Ensure the conditions return only one row
        if len(row) == 1:
            filtered_data.loc[x, "Last_Off_Total_YDS"] = row["Total_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Off_Total_YDS/G"] = row["Total_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Off_Passing_YDS"] = row["Passing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Off_Passing_YDS/G"] = row["Passing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Off_Rushing_YDS"] = row["Rushing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Off_Rushing_YDS/G"] = row["Rushing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Off_Points_PTS"] = row["Points_PTS"].iloc[0]
            filtered_data.loc[x, "Last_Off_PointsPTS/G"] = row["PointsPTS/G"].iloc[0]

In [193]:
def def_scoring_creation(filtered_data):
    filtered_data["Last_Def_Total_YDS"] = 0
    filtered_data["Last_Def_Total_YDS/G"] = 0
    filtered_data["Last_Def_Passing_YDS"] = 0
    filtered_data["Last_Def_Passing_YDS/G"] = 0
    filtered_data["Last_Def_Rushing_YDS"] = 0
    filtered_data["Last_Def_Rushing_YDS/G"] = 0
    filtered_data["Last_Def_Points_PTS"] = 0
    filtered_data["Last_Def_PointsPTS/G"] = 0
    for x in filtered_data.index:
        Team = filtered_data.loc[x]["Tm"]
        Year = filtered_data.loc[x]["Year"]
        row = Defensive_Team_Stats[(Defensive_Team_Stats["TEAM"] == Team) & (Defensive_Team_Stats["Year"] == Year)]

        # Ensure the conditions return only one row
        if len(row) == 1:
            filtered_data.loc[x, "Last_Def_Total_YDS"] = row["Total_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Def_Total_YDS/G"] = row["Total_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Def_Passing_YDS"] = row["Passing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Def_Passing_YDS/G"] = row["Passing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Def_Rushing_YDS"] = row["Rushing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Def_Rushing_YDS/G"] = row["Rushing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Def_Points_PTS"] = row["Points_PTS"].iloc[0]
            filtered_data.loc[x, "Last_Def_PointsPTS/G"] = row["PointsPTS/G"].iloc[0]

In [194]:
#delete duplicate rows
filtered_data = filtered_data.drop_duplicates(keep='last')

In [195]:
def_scoring_creation(filtered_data)
offensive_scoring_creation(filtered_data)

In [196]:
def wr_week_one(data, team, opp, players, year):
    
    summary_team_stats = pd.DataFrame(data={"Last_Off_Total_YDS" : [0], 
                                            "Last_Off_Total_YDS/G" : [0], 
                                            "Last_Off_Passing_YDS" : [0],
                                            "Last_Off_Passing_YDS/G" : [0], 
                                            "Last_Off_Rushing_YDS" : [0], 
                                            "Last_Off_Rushing_YDS/G" : [0], 
                                            "Last_Off_Points_PTS" : [0], 
                                            "Last_Off_PointsPTS/G" : [0], 
                                            "Last_Def_Total_YDS" : [0],
                                            "Last_Def_Total_YDS/G" : [0], 
                                            "Last_Def_Passing_YDS" : [0], 
                                            "Last_Def_Passing_YDS/G" : [0], 
                                            "Last_Def_Rushing_YDS" : [0], 
                                            "Last_Def_Rushing_YDS/G" : [0], 
                                            "Last_Def_Points_PTS" : [0],
                                            "Last_Def_PointsPTS/G" : [0]})
    
    temp = pd.DataFrame(data = {"Name_":players})
    drop = ["Name_", "Tm", "1", "Opp", "Week", "Year", "Day", 
            'Tie?','Score-not-counted', "Weight(lbs)", "Inches", "Feet",
           "Last_Off_Total_YDS", "Last_Off_Total_YDS/G", "Last_Off_Passing_YDS",
           "Last_Off_Passing_YDS/G", "Last_Off_Rushing_YDS", "Last_Off_Rushing_YDS/G", 
           "Last_Off_Points_PTS", "Last_Off_PointsPTS/G", "Last_Def_Total_YDS",
           "Last_Def_Total_YDS/G", "Last_Def_Passing_YDS", "Last_Def_Passing_YDS/G", 
           "Last_Def_Rushing_YDS", "Last_Def_Rushing_YDS/G", "Last_Def_Points_PTS",
           "Last_Def_PointsPTS/G"]
    #temp = data.drop(columns = ["Name_", "Tm", "1", "Opp"])
    used_data = data.drop(columns = drop)
    
    for col in used_data.columns:
            if used_data[col].dtype != object:
                #temp[col + "_Count"] = 0
                temp[col + "_Average"] = 0
                temp[col + "_STD"] = 0
                temp[col + "_Min"] = 0
                temp[col + "_25%"] = 0
                temp[col + "_50%"] = 0
                temp[col + "_100%"] = 0
                temp[col + "_Max"] = 0
                
                
                #temp["History_Opp_" + col + "_Count"] = 0
                temp["History_" + col + "_Average"] = 0
                temp["History_" + col + "_STD"] = 0
                temp["History_" + col + "_Min"] = 0
                temp["History_" + col + "_25%"] = 0
                temp["History_" + col + "_50%"] = 0
                temp["History_" + col + "_100%"] = 0
                temp["History_" + col + "_Max"] = 0
    
    
    
    seperate_df = list(summary_team_stats.columns)
    
    temp["Feet"] = 0
    temp["Inches"] = 0
    temp["Weight(lbs)"] = 0
    temp["Actaul_Dk_Points"] = 0    
                
    sub_data = data[data["Year"] > 2013]
    #History of all games per player on team   
    index = 0
    for name in players:
        indiv_player = data[(data["Name_"] == name) & (data["Year"] < year)]
        indiv_player = indiv_player.drop(columns = drop)
       
        
        for col in indiv_player:
            if seperate_df.count(col) == 0:
                stats = indiv_player[col].describe()
                #temp[col + "_Count"][index] = stats[0]
                temp[col + "_Average"][index] = stats[1]
                temp[col + "_STD"][index] = stats[2]
                temp[col + "_Min"][index] = stats[3]
                temp[col + "_25%"][index] = stats[4]
                temp[col + "_50%"][index] = stats[5]
                temp[col + "_100%"][index] = stats[6]
                temp[col + "_Max"][index] = stats[7]   
        index += 1
    
    
    #History of all players on team vs oppenant
    index = 0 
    for name in players:
        last_games = data[(data["Name_"] == name) & (data["Opp"] == opp) & (data["Year"] < year)]
        last_games = last_games.drop(columns = drop)
        
        for col in last_games:
            if seperate_df.count(col) == 0:
                stats = last_games[col].describe()
                #temp["History_Opp_" + col + "_Count"][index] = stats[0]
                temp["History_" + col + "_Average"][index] = stats[1]
                temp["History_" + col + "_STD"][index] = stats[2]
                temp["History_" + col + "_Min"][index] = stats[3]
                temp["History_" + col + "_25%"][index] = stats[4]
                temp["History_" + col + "_50%"][index] = stats[5]
                temp["History_" + col + "_100%"][index] = stats[6]
                temp["History_" + col + "_Max"][index] = stats[7]   
        index += 1
    
    temp["Opp"] = opp
    temp["Team"] = team
    
    
    
    
    #Get the real points they made and bio
    index = 0
    for name in players:
        
        test_game = data[(data["Name_"] == name) & (data["Opp"] == opp) & (data["Year"] == year) & (data["Week"] == 1)]

        # Assuming test_game contains only one row, otherwise this will error out.
        if len(test_game) == 1:
            #print(test_game["Dk_Points"].iloc[0])

            temp.loc[index, "Actaul_Dk_Points"] = test_game["Dk_Points"].iloc[0]
            temp.loc[index, "Feet"] = test_game["Feet"].iloc[0]
            temp.loc[index, "Inches"] = test_game["Inches"].iloc[0]
            temp.loc[index, "Weight(lbs)"] = test_game["Weight(lbs)"].iloc[0]
        else:
            # Handle cases where test_game might have more than one row or no rows.
            print(f"Expected only one row for conditions, but found {len(test_game)} rows.")
        
        index += 1
    print(test_game["Last_Off_Total_YDS"])
    summary_team_stats["Last_Off_Total_YDS"] = float(test_game["Last_Off_Total_YDS"])
    summary_team_stats["Last_Off_Total_YDS/G"] = float(test_game["Last_Off_Total_YDS"])
    summary_team_stats["Last_Off_Passing_YDS"] = float(test_game["Last_Off_Passing_YDS"])
    summary_team_stats["Last_Off_Passing_YDS/G"] = float(test_game["Last_Off_Passing_YDS/G"])
    summary_team_stats["Last_Off_Rushing_YDS"] = float(test_game["Last_Off_Rushing_YDS"])
    summary_team_stats["Last_Off_Rushing_YDS/G"] = float(test_game["Last_Off_Rushing_YDS/G"])
    summary_team_stats["Last_Off_Points_PTS"] = float(test_game["Last_Off_Points_PTS"])
    summary_team_stats["Last_Off_PointsPTS/G"] = float(test_game["Last_Off_PointsPTS/G"])
    
    summary_team_stats["Last_Def_Total_YDS"] = float(test_game["Last_Def_Total_YDS"])
    summary_team_stats["Last_Def_Total_YDS/G"] = float(test_game["Last_Def_Total_YDS/G"])
    summary_team_stats["Last_Def_Passing_YDS"] = float(test_game["Last_Def_Passing_YDS"])
    summary_team_stats["Last_Def_Passing_YDS/G"] = float(test_game["Last_Def_Passing_YDS/G"])
    summary_team_stats["Last_Def_Rushing_YDS"] = float(test_game["Last_Def_Rushing_YDS"])
    summary_team_stats["Last_Def_Rushing_YDS/G"] = float(test_game["Last_Def_Rushing_YDS/G"])
    summary_team_stats["Last_Def_Points_PTS"] = float(test_game["Last_Def_Points_PTS"])
    summary_team_stats["Last_Def_PointsPTS/G"] = float(test_game["Last_Def_PointsPTS/G"])
        
        
        
        
        
    
    temp = temp.fillna(0)
    
    
    return temp, summary_team_stats

In [197]:
ready_data, team_data = wr_week_one(filtered_data, "CIN", "PIT", ["Kelce, Travis"], year = 2022)

Expected only one row for conditions, but found 0 rows.
Series([], Name: Last_Off_Total_YDS, dtype: int64)


TypeError: cannot convert the series to <class 'float'>

In [182]:
ready_data

,Name_,Off. Snaps_Num_Average,Off. Snaps_Num_STD,Off. Snaps_Num_Min,Off. Snaps_Num_25%,Off. Snaps_Num_50%,Off. Snaps_Num_100%,Off. Snaps_Num_Max,History_Off. Snaps_Num_Average,History_Off. Snaps_Num_STD,History_Off. Snaps_Num_Min,History_Off. Snaps_Num_25%,History_Off. Snaps_Num_50%,History_Off. Snaps_Num_100%,History_Off. Snaps_Num_Max,Receiving_Rec_Average,Receiving_Rec_STD,Receiving_Rec_Min,Receiving_Rec_25%,Receiving_Rec_50%,Receiving_Rec_100%,Receiving_Rec_Max,History_Receiving_Rec_Average,History_Receiving_Rec_STD,History_Receiving_Rec_Min,History_Receiving_Rec_25%,History_Receiving_Rec_50%,History_Receiving_Rec_100%,History_Receiving_Rec_Max,Receiving_Tgt_Average,Receiving_Tgt_STD,Receiving_Tgt_Min,Receiving_Tgt_25%,Receiving_Tgt_50%,Receiving_Tgt_100%,Receiving_Tgt_Max,History_Receiving_Tgt_Average,History_Receiving_Tgt_STD,History_Receiving_Tgt_Min,History_Receiving_Tgt_25%,History_Receiving_Tgt_50%,History_Receiving_Tgt_100%,History_Receiving_Tgt_Max,Receiving_Y/R_Average,Receiving_Y/R_STD,Receiving_Y/R_Min,Receiving_Y/R_25%,Receiving_Y/R_50%,Receiving_Y/R_100%,Receiving_Y/R_Max,History_Receiving_Y/R_Average,History_Receiving_Y/R_STD,History_Receiving_Y/R_Min,History_Receiving_Y/R_25%,History_Receiving_Y/R_50%,History_Receiving_Y/R_100%,History_Receiving_Y/R_Max,Receiving_Y/Tgt_Average,Receiving_Y/Tgt_STD,Receiving_Y/Tgt_Min,Receiving_Y/Tgt_25%,Receiving_Y/Tgt_50%,Receiving_Y/Tgt_100%,Receiving_Y/Tgt_Max,History_Receiving_Y/Tgt_Average,History_Receiving_Y/Tgt_STD,History_Receiving_Y/Tgt_Min,History_Receiving_Y/Tgt_25%,History_Receiving_Y/Tgt_50%,History_Receiving_Y/Tgt_100%,History_Receiving_Y/Tgt_Max,Receiving_Yds_Average,Receiving_Yds_STD,Receiving_Yds_Min,Receiving_Yds_25%,Receiving_Yds_50%,Receiving_Yds_100%,Receiving_Yds_Max,History_Receiving_Yds_Average,History_Receiving_Yds_STD,History_Receiving_Yds_Min,History_Receiving_Yds_25%,History_Receiving_Yds_50%,History_Receiving_Yds_100%,History_Receiving_Yds_Max,ST Snaps_Num_Average,ST Snaps_Num_STD,ST Snaps_Num_Min,ST Snaps_Num_25%,ST Snaps_Num_50%,ST Snaps_Num_100%,ST Snaps_Num_Max,History_ST Snaps_Num_Average,History_ST Snaps_Num_STD,History_ST Snaps_Num_Min,History_ST Snaps_Num_25%,History_ST Snaps_Num_50%,History_ST Snaps_Num_100%,History_ST Snaps_Num_Max,Age_Average,Age_STD,Age_Min,Age_25%,Age_50%,Age_100%,Age_Max,History_Age_Average,History_Age_STD,History_Age_Min,History_Age_25%,History_Age_50%,History_Age_100%,History_Age_Max,COVID-19 List_Average,COVID-19 List_STD,COVID-19 List_Min,COVID-19 List_25%,COVID-19 List_50%,COVID-19 List_100%,COVID-19 List_Max,History_COVID-19 List_Average,History_COVID-19 List_STD,History_COVID-19 List_Min,History_COVID-19 List_25%,History_COVID-19 List_50%,History_COVID-19 List_100%,History_COVID-19 List_Max,Physically Unable to Perform_Average,Physically Unable to Perform_STD,Physically Unable to Perform_Min,Physically Unable to Perform_25%,Physically Unable to Perform_50%,Physically Unable to Perform_100%,Physically Unable to Perform_Max,History_Physically Unable to Perform_Average,History_Physically Unable to Perform_STD,History_Physically Unable to Perform_Min,History_Physically Unable to Perform_25%,History_Physically Unable to Perform_50%,History_Physically Unable to Perform_100%,History_Physically Unable to Perform_Max,Did Not Play_Average,Did Not Play_STD,Did Not Play_Min,Did Not Play_25%,Did Not Play_50%,Did Not Play_100%,Did Not Play_Max,History_Did Not Play_Average,History_Did Not Play_STD,History_Did Not Play_Min,History_Did Not Play_25%,History_Did Not Play_50%,History_Did Not Play_100%,History_Did Not Play_Max,Exempt List_Average,Exempt List_STD,Exempt List_Min,Exempt List_25%,Exempt List_50%,Exempt List_100%,Exempt List_Max,History_Exempt List_Average,History_Exempt List_STD,History_Exempt List_Min,History_Exempt List_25%,History_Exempt List_50%,History_Exempt List_100%,History_Exempt List_Max,Injured Reserve_Average,Injured Reserve_STD,Injured Reserve_Min,Injured Reserve_25%,Injured Reserve_50%,Injured Reserve_1

In [183]:
team_data

,Last_Off_Total_YDS,Last_Off_Total_YDS/G,Last_Off_Passing_YDS,Last_Off_Passing_YDS/G,Last_Off_Rushing_YDS,Last_Off_Rushing_YDS/G,Last_Off_Points_PTS,Last_Off_PointsPTS/G,Last_Def_Total_YDS,Last_Def_Total_YDS/G,Last_Def_Passing_YDS,Last_Def_Passing_YDS/G,Last_Def_Rushing_YDS,Last_Def_Rushing_YDS/G,Last_Def_Points_PTS,Last_Def_PointsPTS/G
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
filtered_data[(filtered_data["Name_"] == "Perine, Samaje") & (filtered_data["Year"] == 2022)]

,Name_,Off. Snaps_Num,Receiving_Rec,Receiving_Tgt,Receiving_Y/R,Receiving_Y/Tgt,Receiving_Yds,ST Snaps_Num,Year,Week,Age,Tm,1,Opp,COVID-19 List,Physically Unable to Perform,Did Not Play,Exempt List,Injured Reserve,Non-Football Injury,*,Suspended,Inactive,Win?,Lose?,Tie?,Score-not-counted,Tm-Score,Opp-Score,Day,Month,Dk_Points,Salary,Feet,Inches,Weight(lbs),Last_Def_Total_YDS,Last_Def_Total_YDS/G,Last_Def_Passing_YDS,Last_Def_Passing_YDS/G,Last_Def_Rushing_YDS,Last_Def_Rushing_YDS/G,Last_Def_Points_PTS,Last_Def_PointsPTS/G,Last_Off_Total_YDS,Last_Off_Total_YDS/G,Last_Off_Passing_YDS,Last_Off_Passing_YDS/G,Last_Off_Rushing_YDS,Last_Off_Rushing_YDS/G,Last_Off_Points_PTS,Last_Off_PointsPTS/G
